In [ ]:
import subprocess
import sys

def install_packages():
    """Install all required packages"""
    packages = [
        "qiskit==0.45.0",
        "qiskit-machine-learning==0.7.0", 
        "qiskit-algorithms==0.2.0",
        "opencv-python",
        "scikit-learn",
        "matplotlib",
        "seaborn",
        "Pillow"
    ]
    
    print("Installing packages...")
    for package in packages:
        print(f"Installing {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ {package} installed successfully")
        except subprocess.CalledProcessError as e:
            print(f"✗ Failed to install {package}: {e}")
    
    print("\n" + "="*50)
    print("INSTALLATION COMPLETE!")
    print("IMPORTANT: Please restart the runtime now!")
    print("Go to Runtime → Restart Runtime")
    print("="*50)

install_packages()

import os
os.kill(os.getpid(), 9)

Installing packages...
Installing qiskit==0.45.0...
✓ qiskit==0.45.0 installed successfully
Installing qiskit-machine-learning==0.7.0...
✓ qiskit-machine-learning==0.7.0 installed successfully
Installing qiskit-algorithms==0.2.0...
✓ qiskit-algorithms==0.2.0 installed successfully
Installing opencv-python...
✓ opencv-python installed successfully
Installing scikit-learn...
✓ scikit-learn installed successfully
Installing matplotlib...
✓ matplotlib installed successfully
Installing seaborn...
✓ seaborn installed successfully
Installing Pillow...


: 

: 

: 

In [12]:
pip install qiskit.quantum_info

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement qiskit.quantum_info (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Vansh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for qiskit.quantum_info


In [4]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.circuit import ParameterVector
from qiskit.primitives import Estimator
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import ZZFeatureMap
from qiskit_algorithms.optimizers import SPSA

algorithm_globals.random_seed = 42


class QuantumHumanDetector:
    def __init__(self, feature_dim=16, num_qubits=6):
        self.feature_dim = feature_dim
        self.num_qubits = num_qubits
        self.scaler = MinMaxScaler(feature_range=(0, np.pi))
        self.qnn = None
        self.classifier = None
        self.training_history = []
        print(f'Using Quantum Neural Network with {num_qubits} qubits and {feature_dim} features')

    def load_and_preprocess_data(self, dataset_path):
        print("Loading and preprocessing dataset...")
        images, labels = [], []
        for class_label in ['0', '1']:
            class_path = os.path.join(dataset_path, class_label)
            if os.path.exists(class_path):
                for filename in os.listdir(class_path):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(class_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            images.append(img)
                            labels.append(int(class_label))

        print(f"Loaded {len(images)} images")
        if len(images) > 0:
            print(f"Class distribution: {np.bincount(labels)}")

        features = self._extract_features(images)
        return np.array(features), np.array(labels)

    def _extract_features(self, images):
        features = []
        for img in images:
            if len(img.shape) == 3:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray = img

            resized = cv2.resize(gray, (128, 128))
            feature_vector = []
            hist = cv2.calcHist([resized], [0], None, [8], [0, 256])
            feature_vector.extend(hist.flatten())

            kernel = np.ones((3, 3), np.uint8)
            mean_filtered = cv2.filter2D(resized, -1, kernel / 9)
            texture_contrast = np.std(resized - mean_filtered)
            feature_vector.append(texture_contrast)

            edges = cv2.Canny(resized, 50, 150)
            edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
            feature_vector.append(edge_density)

            grad_x = cv2.Sobel(resized, cv2.CV_64F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(resized, cv2.CV_64F, 0, 1, ksize=3)
            grad_magnitude = np.sqrt(grad_x ** 2 + grad_y ** 2)
            feature_vector.append(np.mean(grad_magnitude))
            feature_vector.append(np.std(grad_magnitude))

            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                feature_vector.append(len(contours))
                max_area = max([cv2.contourArea(c) for c in contours])
                feature_vector.append(max_area / (resized.shape[0] * resized.shape[1]))
            else:
                feature_vector.extend([0, 0])

            f_transform = np.fft.fft2(resized)
            f_shift = np.fft.fftshift(f_transform)
            magnitude_spectrum = np.abs(f_shift)
            feature_vector.append(np.mean(magnitude_spectrum))
            feature_vector.append(np.std(magnitude_spectrum))

            feature_vector.append(np.mean(resized))
            feature_vector.append(np.std(resized))
            feature_vector.append(np.var(resized))

            if len(feature_vector) < self.feature_dim:
                feature_vector.extend([0] * (self.feature_dim - len(feature_vector)))
            else:
                feature_vector = feature_vector[:self.feature_dim]

            features.append(feature_vector)
        return np.array(features)
    
    

    def create_quantum_circuit(self):
        print("Creating quantum circuit with matched input and parameter dimensions...")

        # Feature map that expects input dimension equal to num_qubits
        feature_map = ZZFeatureMap(
            feature_dimension=self.num_qubits,
            reps=2,
            entanglement='full',
            insert_barriers=True
        )

        ansatz = RealAmplitudes(
            num_qubits=self.num_qubits,
            reps=2,
            entanglement='full',
            insert_barriers=True
        )

        input_params = list(feature_map.parameters)
        weight_params = list(ansatz.parameters)

        qc = QuantumCircuit(self.num_qubits)
        qc.compose(feature_map, inplace=True)
        qc.compose(ansatz, inplace=True)

        estimator = Estimator()
        observable = SparsePauliOp.from_list([("Z" + "I" * (self.num_qubits - 1), 1.0)])

        self.qnn = EstimatorQNN(
            circuit=qc,
            input_params=input_params,
            weight_params=weight_params,
            estimator=estimator,
            observables=observable
        )

        return qc





    def train(self, X, y, test_size=0.2, max_iter=50):
        print("Training quantum neural network...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        if X_train_scaled.shape[1] < self.num_qubits:
            padding = np.zeros((X_train_scaled.shape[0], self.num_qubits - X_train_scaled.shape[1]))
            X_train_scaled = np.hstack([X_train_scaled, padding])
            padding = np.zeros((X_test_scaled.shape[0], self.num_qubits - X_test_scaled.shape[1]))
            X_test_scaled = np.hstack([X_test_scaled, padding])
        else:
            X_train_scaled = X_train_scaled[:, :self.num_qubits]
            X_test_scaled = X_test_scaled[:, :self.num_qubits]

        self.X_test = X_test_scaled
        self.y_test = y_test

        self.create_quantum_circuit()
        optimizer = SPSA(maxiter=max_iter)

        def callback(weights, loss):
            self.training_history.append(loss)
            if len(self.training_history) % 10 == 0:
                print(f"Iteration {len(self.training_history)}: Loss = {loss:.4f}")

        self.classifier = NeuralNetworkClassifier(
            neural_network=self.qnn,
            optimizer=optimizer,
            callback=callback
        )

        print("Starting quantum training...")
        self.classifier.fit(X_train_scaled, y_train)
        y_pred = self.classifier.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"\nQuantum Model Test Accuracy: {accuracy:.4f}")
        return accuracy


def main():
    print("Quantum Human Detection")
    print("=" * 60)

    detector = QuantumHumanDetector(feature_dim=16, num_qubits=6)
    dataset_path = r"C:\\Users\\Vansh\\OneDrive\\Documents\\My Games\\human_detection_dataset"

    X, y = detector.load_and_preprocess_data(dataset_path)

    if len(X) == 0:
        print("No data loaded! Please check your dataset path.")
        return None

    # Classical Baseline
    print("\nTraining Classical Baseline...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    classical_acc = accuracy_score(y_test, y_pred)
    print(f"Classical Model Accuracy: {classical_acc:.4f}")

    # Quantum Model Training
    accuracy = detector.train(X, y, max_iter=50)

    return detector


if __name__ == "__main__":
    detector = main()


Quantum Human Detection
Using Quantum Neural Network with 6 qubits and 16 features
Loading and preprocessing dataset...
Loaded 921 images
Class distribution: [362 559]

Training Classical Baseline...
Classical Model Accuracy: 0.5892
Training quantum neural network...
Creating quantum circuit with matched input and parameter dimensions...
Starting quantum training...
Iteration 10: Loss = 0.5877
Iteration 20: Loss = 0.5961
Iteration 30: Loss = 0.5735
Iteration 40: Loss = 0.5668
Iteration 50: Loss = 0.5915
Iteration 60: Loss = 0.5509
Iteration 70: Loss = 0.5304
Iteration 80: Loss = 0.5290
Iteration 90: Loss = 0.5203
Iteration 100: Loss = 0.5177
Iteration 110: Loss = 0.5167
Iteration 120: Loss = 0.5197
Iteration 130: Loss = 0.5203
Iteration 140: Loss = 0.5177
Iteration 150: Loss = 0.5163

Quantum Model Test Accuracy: 0.4270
